In [1]:
import zipfile
with zipfile.ZipFile("/content/drive/My Drive/SA.zip",'r') as f:
    f.extractall('')

In [2]:
import pandas as pd
df1=pd.read_csv('/content/SA/test_data.csv')

In [3]:
df1.head()

,test_id,text
0,test_1,trying to have a nice quiet dinner. the annou...
1,test_2,Been getting food to go from here for over 3yr...
2,test_3,Ugh. I've had to eat here a couple of times be...
3,test_4,The people here are so nice! I ordered on eat ...
4,test_5,Heard alot of good things about this place and...


In [4]:
df2=pd.read_csv('/content/SA/train_data.csv')
df3=pd.read_csv('/content/SA/train_label.csv')
df2.head()

,trn_id,text
0,trn_1,Well this place got me to write my first revie...
1,trn_2,A very good Greek restaurant with tasty food. ...
2,trn_3,"Website says open, Google says open, Yelp says..."
3,trn_4,If I could give zero stars I would. When we wa...
4,trn_5,They have great food & definitely excellent se...


In [5]:
df3=df3.drop(['trn_id'], axis=1)

In [6]:
df3 = pd.concat([df2, df3], axis = 1) 
df3.head()

,trn_id,text,label
0,trn_1,Well this place got me to write my first revie...,2
1,trn_2,A very good Greek restaurant with tasty food. ...,5
2,trn_3,"Website says open, Google says open, Yelp says...",1
3,trn_4,If I could give zero stars I would. When we wa...,1
4,trn_5,They have great food & definitely excellent se...,5


In [7]:
df3=df3.drop(['trn_id'], axis=1)

In [8]:
df3['label'].unique()

array([2, 5, 1, 4, 3])

In [9]:
df3.head()

,text,label
0,Well this place got me to write my first revie...,2
1,A very good Greek restaurant with tasty food. ...,5
2,"Website says open, Google says open, Yelp says...",1
3,If I could give zero stars I would. When we wa...,1
4,They have great food & definitely excellent se...,5


In [10]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df3['label'] = labelencoder.fit_transform(df3['label'])

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding,SpatialDropout1D
import re

In [12]:
df3['text']=df3['text'].apply(lambda x:x.lower())
print(df3['text'].iloc[0])
df3['text']=df3['text'].apply(lambda x:re.sub('[^a-zA-Z0-9\s]','',x))
print(df3['text'].iloc[0])

well this place got me to write my first review. with all the yelp reviews i figured it would be a good taco spot (and the 4-5 dollar price range per taco) so my wife and i decided to stop by. usually a place that has that price range like puesto in san diego or several taco food trucks in la they have amazing tacos. maybe it was a bad day but the steak and the chicken tacos i had were way over cooked and honestly only tasted like the sauce you chose to put on in. the add on for rice and beans were 2 dollars which got you a ketchup cup of each lol my wife's tacos were a little bit better though. the corn tortillas weren't the greatest either
well this place got me to write my first review with all the yelp reviews i figured it would be a good taco spot and the 45 dollar price range per taco so my wife and i decided to stop by usually a place that has that price range like puesto in san diego or several taco food trucks in la they have amazing tacos maybe it was a bad day but the steak 

In [13]:
max_f=2000
token=Tokenizer(num_words=max_f,split=' ')
token.fit_on_texts(df3['text'].values)
seq = token.texts_to_sequences(df3['text'].values)

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
seq=pad_sequences(seq)

In [15]:
x_train=seq
y_train=df3.iloc[:,1].values

In [16]:
y_train.shape

(650000,)

In [17]:
df1=df1.drop(['test_id'], axis=1)

In [18]:
df1['text']=df1['text'].apply(lambda x:x.lower())
df1['text']=df1['text'].apply(lambda x:re.sub('[^a-zA-Z0-9\s]','',x))

In [19]:
embed_dim=140

In [20]:
model = Sequential()
model.add(Embedding(max_f,embed_dim,input_length=seq.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

In [21]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,batch_size=50,epochs=10,verbose=1)

Epoch 1/10
 1922/13000 [===>..........................] - ETA: 4:10:36 - loss: -133.7788 - accuracy: 0.1968